In [1]:
import sys; sys.path.append("../../../../automl/")

In [2]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model.lama import TabularLama
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-08 13:02:50]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

In [3]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [4]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../../data/")
CONFIG_PATH = Path("../../../../configs/config.yaml")
N_JOBS = 16

In [5]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [6]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_oof.parquet")
#df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [7]:
df_train["target"].value_counts(normalize=True)

0    0.940982
1    0.059018
Name: target, dtype: float64

In [8]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [14]:
cat_columns = []

In [15]:
X_train, y_train = df_train[cfg["stack_features"]], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [16]:
display(y_train.value_counts(normalize=True))
#display(y_test.value_counts(normalize=True))

0    0.940982
1    0.059018
Name: target, dtype: float64

In [17]:
#categorical_features = ohe_cols# + oe_cols

## Model

In [18]:
metric = RocAuc()

In [19]:
model = TabularLama(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 30, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-08 13:03:14,115] - [   START    ] - Fitting TabularLama
[13:03:14] Stdout logging level is INFO.
[13:03:14] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[13:03:14] Task: binary

[13:03:14] Start automl preset with listed constraints:
[13:03:14] - time: 3600.00 seconds
[13:03:14] - CPU: 16 cores
[13:03:14] - memory: 16 GB

[13:03:14] Train data shape: (413194, 8)

[13:03:15] Layer 1 train process start. Time left 3598.60 secs
[13:03:15] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[13:03:17] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8087671381703095
[13:03:17] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[13:03:17] Time left 3596.76 secs

[13:03:21] Selector_LightGBM fitting and predicting completed
[13:03:21] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[13:03:44] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8119925394963108
[13:03:44] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predic

In [14]:
model = TabularLama(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 30, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-08 09:07:53,431] - [   START    ] - Fitting TabularLama
[09:07:53] Stdout logging level is INFO.
[09:07:53] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[09:07:53] Task: binary

[09:07:53] Start automl preset with listed constraints:
[09:07:53] - time: 3600.00 seconds
[09:07:53] - CPU: 16 cores
[09:07:53] - memory: 16 GB

[09:07:53] Train data shape: (413194, 21)

[09:07:57] Layer 1 train process start. Time left 3596.00 secs
[09:07:57] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[09:08:00] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8082552247714897
[09:08:00] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[09:08:00] Time left 3592.98 secs

[09:08:05] Selector_LightGBM fitting and predicting completed
[09:08:05] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[09:08:31] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8037248750001251
[09:08:31] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predi

In [21]:
model = TabularLama(n_jobs=16, task="classification")
model.tune(X_train, y_train, metric, timeout=60 * 30, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 13:50:21,142] - [   START    ] - Fitting TabularLama
[13:50:21] Stdout logging level is INFO.
[13:50:21] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[13:50:21] Task: binary

[13:50:21] Start automl preset with listed constraints:
[13:50:21] - time: 3600.00 seconds
[13:50:21] - CPU: 16 cores
[13:50:21] - memory: 16 GB

[13:50:21] Train data shape: (413194, 68)

[13:50:34] Layer 1 train process start. Time left 3586.87 secs
[13:50:48] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[13:51:42] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.8078772570316353
[13:51:42] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[13:51:42] Time left 3518.73 secs

[13:51:49] Selector_LightGBM fitting and predicting completed
[13:52:01] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[13:52:35] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8085887315347171
[13:52:35] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predi

In [23]:
MODEL_NAME = "lama_stack_8136_full_dataset"
MODEL_DIR = Path(f"../../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [16]:
# res = pd.DataFrame()
# res[MODEL_NAME] = oof[:, 1]
# res.to_csv(MODEL_DIR / "oof.csv", index=False)
# #joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

# with (MODEL_DIR / "params.yaml").open("w") as f:
#     yaml.dump(model.params, f)

# with (MODEL_DIR / "score.txt").open("w") as f:
#     print("OOF:", metric(y_train, oof), file=f)
    
# test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
# test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
# test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [24]:
test = pd.read_parquet(DATA_PATH / "test_preproc_oof.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cfg["stack_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(f'{MODEL_NAME}.csv', index=False)

In [26]:
model.model.get_feature_scores()[:50]

,Feature,Importance
0,lamau_814_full_dataset,78048.337071
1,xgb_81325_full_dataset,29297.001751
2,lama_81298_full_dataset,24098.444188
3,feature_162,1398.695796
4,feature_24,1321.674997
5,feature_18,1298.269506
6,feature_26,1290.120701
7,feature_145,1287.683197
8,feature_78,1279.916003
9,feature_36,1248.758602


In [24]:
imp = pd.DataFrame().assign(names=model.models[0].feature_names_, imp=model.models[0].feature_importances_)

In [41]:
imp.sort_values(by="imp", ascending=False).reset_index(drop=True).query("names == 'feature_185'")

,names,imp
61,feature_185,0.0


In [29]:
cat_columns

['feature_7',
 'feature_31',
 'feature_60',
 'feature_61',
 'feature_71',
 'feature_109',
 'feature_122',
 'feature_156',
 'feature_163',
 'feature_167',
 'feature_179',
 'feature_185']

## With Time series cross val

In [13]:
metric = RocAuc()

In [14]:
df_train = df_train.sort_values(by="id").reset_index(drop=True)
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

In [15]:
model = LightGBMClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 08:27:33,388] - [   START    ] - Tuning LightGBMClassification
[2024-11-07 08:27:40,118] - [   OPTUNA   ] - Trial 0. New best score 0.7903405446081995 with parameters {'max_depth': 6, 'num_leaves': 488, 'min_data_in_leaf': 188, 'bagging_fraction': 0.7993292420985183, 'bagging_freq': 0, 'feature_fraction': 0.49359671220172163, 'lambda_l1': 0.5808361216819946, 'lambda_l2': 8.661761457749352, 'min_gain_to_split': 12.022300234864176, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:03,043] - [   OPTUNA   ] - Trial 2. New best score 0.7913515589848906 with parameters {'max_depth': 5, 'num_leaves': 194, 'min_data_in_leaf': 117, 'bagging_fraction': 0.8925879806965068, 'bagging_freq': 0, 'feature_fraction': 0.708540663048167, 'lambda_l1': 5.924145688620425, 'lambda_l2': 0.46450412719997725, 'min_gain_to_split': 12.150897038028766, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:12,225] - [   OPTUNA   ] - Trial 3. New best score 0.8048686053278628 with param

In [27]:
none_oofs_idx = oof[np.any(np.isnan(oof), axis=1)].shape[0]

In [32]:
metric(y_train[none_oofs_idx:], oof[none_oofs_idx:])

0.8095227594190041

In [34]:
MODEL_NAME = "lgb_8095_full_dataset_time_series"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [35]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[none_oofs_idx:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## TEST 
**81.22112399468679**

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method